<h1 align="center">Primeiros passos com a Prompt Engineering</h1>

Data Scientist Jr.: Karina Gonçalves Soares

Links de estudo:

* [Prompt-Engineering-Guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/notebooks/pe-lecture.ipynb)
* [O que é Engenharia de Prompt...](https://www.alura.com.br/artigos/engenharia-prompt)


por DAIR.AI | Elvis Saravia

    Este caderno contém exemplos e exercícios para aprender sobre Prompt Engineering.

    Usaremos as APIs OpenAI para todos os exemplos. Estou usando as configurações padrão de temperature = 0,7 e top-p = 1

1. Noções básicas de Prompt Engineering

* Objetivos

    * Carregue as bibliotecas
    * Revise o formato
    * Cubra os prompts básicos
    * Revise casos de uso comuns

Abaixo estamos carregando as bibliotecas, utilitários e configurações necessárias.

In [1]:
import openai # Permite interação com modelos de linguagem treinados pela OpenAI
import os
import IPython # Recursos para formatação
from langchain.llms import OpenAI
from dotenv import load_dotenv, find_dotenv

In [2]:
_ = load_dotenv(find_dotenv()) # read lical ,env file
key_openai = os.environ['OPENAI_API_KEY']

In [3]:
def set_open_params(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=256,
    top_p=1, # evita geração de texto irrelevante
    frequency_penalty=0, # penalidade para tokens repetidos
    presence_penalty=0,  # penalidade por tokens que não apareceram
):
    """ definir parâmetros openai"""
    
    openai_params = {}
    
    
    openai_params["model"] = model
    openai_params["temperature"] = temperature
    openai_params["max_tokens"] = max_tokens
    openai_params["top_p"] = top_p
    openai_params["frequency_penalty"] = frequency_penalty
    openai_params["presence_penalty"] = presence_penalty
    return openai_params

def get_completion(params, messages):
    
    """ obter resposta da API openai"""
    
    response = openai.chat.completions.create(
        model = params['model'],
        messages = messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

    
    

Exemplo de prompt básico:

In [4]:
params = set_open_params()

prompt = "qual é a cor do céu?"

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)

In [5]:
response.choices[0].message.content

'A cor do céu varia dependendo das condições climáticas e da hora do dia. Durante o dia, o céu geralmente parece azul devido à dispersão da luz solar pela atmosfera. Durante o pôr do sol e o nascer do sol, o céu pode adquirir tons de laranja, vermelho, rosa e roxo. Durante a noite, o céu costuma ser escuro, mas pode ser iluminado pela lua e pelas estrelas.'

Experimente com temperaturas diferentes para comparar os resultados:

In [6]:
params = set_open_params(temperature=0)
response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

A cor do céu pode variar dependendo da hora do dia e das condições climáticas. Durante o dia, o céu geralmente é azul, devido à dispersão da luz solar na atmosfera. Durante o pôr do sol e o nascer do sol, o céu pode apresentar tons de laranja, rosa, vermelho e roxo. Durante a noite, o céu pode parecer preto, mas na verdade é um tom de azul escuro.

### 1.1 Resumo do Texto

In [7]:
params = set_open_params(temperature=0)

prompt ="""
Responda o texto a seguir levando em consideração que VOCÊ ESTÁ EXPLICANDO PARA UMA CRIANÇA DE 5 ANOS.


Os antibióticos são um tipo de medicamento usado para tratar infecções bacterianas.\n
 Eles atuam matando as bactérias ou impedindo-as de se reproduzirem, permitindo que o sistema imunológico do corpo combata a infecção.\n
 Os antibióticos são geralmente tomados por via oral na forma de comprimidos, cápsulas ou soluções líquidas, ou às vezes administrados por via intravenosa.\n
 Eles não são eficazes contra infecções virais e seu uso inadequado pode levar à resistência aos antibióticos."""
 
 
messages = [
    {
        "role": "user",
        "content": prompt
    }
 ]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Os antibióticos são remédios que ajudam a combater as bactérias que deixam a gente doente. Eles podem matar as bactérias ou impedir que elas se multipliquem, para que nosso corpo consiga se livrar da infecção. 

Os antibióticos podem ser tomados de diferentes formas, como em comprimidos, cápsulas ou até mesmo em líquido. Às vezes, eles também podem ser colocados direto na veia. 

É importante lembrar que os antibióticos não funcionam contra vírus, apenas contra bactérias. E se a gente usar os antibióticos de forma errada, as bactérias podem ficar mais fortes e os remédios podem não funcionar mais. Por isso, é importante sempre seguir as orientações do médico quando precisamos tomar antibióticos.

<font color="red">Exercício:</font>Instrua o modelo a explicar o parágrafo para uma criança de 5 anos. Você vê alguma diferença?

### 1.2 Resposta a perguntas

In [8]:
prompt = """Responda à pergunta com base no contexto abaixo. Mantenha a resposta curta e concisa.\n
Responda "Não tenho certeza sobre a resposta" SE NÃO TIVER CERTEZA DA RESPOSTA.

Contexto: Teplizumab tem suas raízes em uma empresa farmacêutica de Nova Jersey chamada Ortho Pharmaceutical.\n
Lá, os cientistas geraram uma versão inicial do anticorpo, batizada de OKT3.\n
Originalmente proveniente de ratos, a molécula foi capaz de se ligar à superfície das células T e limitar o seu potencial de morte celular.\n
Em 1986, foi aprovado para ajudar a prevenir a rejeição de órgãos após transplantes renais, tornando-se o primeiro anticorpo terapêutico permitido para uso humano.

Pergunta: Qual foi originalmente a origem do OKT3?

Responder:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

O OKT3 foi originalmente proveniente de ratos.

[Contexto obtido **aqui!**](https://www.nature.com/articles/d41586-023-00400-x)

<font color="red">Exercício:</font> Edite o prompt e faça com que o modelo responda que não tem certeza sobre a resposta.

### 1.3 Classificação do Texto

In [9]:
prompt = """
Classifique o texto em neutro, negativo ou positivo e explique o porquê de você ter tirado essa conclusão.

Texto: Acho que a comida estava boa.

Sentimento:
"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Neutro.

Explicação:
O texto expressa uma opinião sobre a comida, mas de forma neutra, sem indicar claramente se a experiência foi positiva ou negativa. A palavra "acho" indica uma opinião pessoal, mas não fornece informações suficientes para determinar se a comida foi considerada boa ou ruim. Portanto, a classificação neutra é a mais apropriada.

<font color="red">Exercício:</font> Modifique o prompt para instruir o modelo a fornecer uma explicação para a resposta selecionada.

### 1.4 Representação de papéis

In [13]:
prompt = """A seguir temos uma conversa entre um Humano e uma Inteligência Artificial. A Inteligência Artificial é assistente em técnica científica.
COMPLETE O DIÁLOGO A SEGRIR TRAZENDO UMA RESPOSTA PARA O HUMANO.

Humano: Olá, quem é você?
IA: Saudação! Sou assistente de pesquisa de IA. Como posso ajudá-lo hoje?
Humano: Você pode me contar sobre a criação de buracos negros?
IA:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Claro! Os buracos negros são regiões do espaço-tempo onde a gravidade é tão intensa que nada, nem mesmo a luz, consegue escapar. Eles são formados a partir do colapso de estrelas massivas no final de sua vida. Essa massa concentrada cria um campo gravitacional extremamente forte, que suga tudo ao seu redor. É como se fosse um poço sem fundo no espaço.

<font color="red">Exercício:</font> Modifique o prompt para instruir o modelo a manter as respostas de IA concisas e curtas.

### 1.5 Geração de Código

In [14]:
prompt = """
Tabela departamentos, colunas = [DepartmentId, DepartmentName]\n
Tabela estudantes, colunas = [DepartmentId, StudentId, StudentName]\n
Crie uma consulta MySQL para todos os alunos do Departamento de Ciência da Computação
"""
messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

SELECT StudentId, StudentName
FROM estudantes
WHERE DepartmentId = (SELECT DepartmentId FROM departamentos WHERE DepartmentName = 'Ciência da Computação');